# Create CG App With Policy

Let's get started!
___

In [ ]:
import pandas as pd, numpy as np, datetime as dt, matplotlib.pyplot as plt
import requests, json
from ipywidgets import widgets, Layout, interact
from IPython.display import display
import base64, urllib.parse

## 1 - Initialize Environment

First let's gather information about your IDCS Tenant.


In [ ]:
tenant = ""
clientId = ''
clientSecret = ''

domain = "identity.oraclecloud.com"
port = "8943"
baseurl = 'https://' + tenant + '.' + domain + ':' + port
configurl =  baseurl + '/.well-known/idcs-configuration'

Specify the Target App Name

In [ ]:
targetappname = ""

## 2 - Generate Access Token


In [ ]:
config = None
response = requests.get(configurl, verify=False).text
config = json.loads(response)
tokenurl = config["openid-configuration"]["token_endpoint"]

grant_type = "client_credentials"
scope = "urn:opc:idm:__myscopes__"

#Token Endpoint needs the App Creds in base64 encoded Header
basicauthHeader = base64.b64encode(bytes(clientId + ":" + clientSecret, 'utf-8')) 
reqdata = {'grant_type': grant_type, 'scope' : scope}
reqheaders = {'Authorization': 'Basic ' + basicauthHeader.decode("utf-8"), 'content-type': 'application/x-www-form-urlencoded'}

response = requests.post(tokenurl, data = reqdata, headers = reqheaders, verify=False)
accesstoken = response.json()["access_token"]
print(accesstoken)


___

## 3 - Search for the App

Search for the target App to place the policy. If does not exist, then create a new App. Get the App ID.

In [ ]:
appsurl = baseurl + "/admin/v1/Apps?filter=displayName eq \"" + targetappname + "\""
targetappid = None

appsheaders = {'Authorization': 'Bearer ' + accesstoken, 'content-type': 'application/json'}
appslist = requests.get(appsurl, headers = appsheaders, verify=False).json()["Resources"]

if len(appslist) == 0:
    print("Creating New Placeholder App")
    
    cgrolevalurl = baseurl + "/admin/v1/AppRoles?filter=displayName eq \"Cloud Gate\""
    cgroleval = requests.get(cgrolevalurl, headers = appsheaders, verify=False).json()["Resources"][0]["id"]
    
    apppayload = {"clientType": "confidential",
        "isAliasApp": False,
        "isLoginTarget": False,
        "displayName": targetappname,
        "isMobileTarget": False,
        "isUnmanagedApp": False,
        "isOPCService": False,
        "isOAuthClient": True,
        "isWebTierPolicy": False,
        "isManagedApp": False,
        "isManagedApp": False,
        "isSamlServiceProvider": False,
        "infrastructure": False,
        "allUrlSchemesAllowed": True,
        "allowAccessControl": False,
        "isOAuthResource": False,
        "migrated": False,
        "logoutUri": "http://%hostid%/cloudgate/v1/oauth2/logout",
        "isKerberosRealm": False,
        "redirectUris": [
            "http://%hostid%/cloudgate/v1/oauth2/callback"
        ],
        "allowedGrants": [
            "client_credentials",
            "authorization_code"
        ],
        "attrRenderingMetadata": [
            {
                "visible": False,
                "name": "aliasApps"
            }
        ],
        "basedOnTemplate": {
            "value": "CustomWebAppTemplateId",
        },
        "schemas": [
            "urn:ietf:params:scim:schemas:oracle:idcs:App",
            "urn:ietf:params:scim:schemas:oracle:idcs:extension:requestable:App",
            "urn:ietf:params:scim:schemas:oracle:idcs:extension:webTierPolicy:App"
        ]
    }
    
    appscreateurl = baseurl + "/admin/v1/Apps"
    response = requests.post(appscreateurl, data = json.dumps(apppayload), headers = appsheaders, verify=False).json()
    print(response)
    targetappid = response["id"]
    
    cggranturl = baseurl + "/admin/v1/Grants"
    grantpayload = {
        "app": {
            "value": "IDCSAppId"
          },
        "entitlement": {
            "attributeName": "appRoles",
            "attributeValue": cgroleval
          },
        "grantMechanism": "ADMINISTRATOR_TO_APP",
        "grantee": {
            "value": targetappid,
            "type": "App"
        },
        "schemas": [
            "urn:ietf:params:scim:schemas:oracle:idcs:Grant"
        ]
    }
    response = requests.post(cggranturl, data = json.dumps(grantpayload), headers = appsheaders, verify=False)
    
    appstatusurl = baseurl + "/admin/v1/AppStatusChanger/" + targetappid
    statuspayload = {
      "active": True,
      "schemas": [
        "urn:ietf:params:scim:schemas:oracle:idcs:AppStatusChanger"
      ]
    }
    response = requests.put(appstatusurl, data = json.dumps(statuspayload), headers = appsheaders, verify=False)
    print(response.text)
    
else:
    targetappid = appslist[0]["id"]

print(targetappid)


___

## 4 - Patch App with Policy

In [ ]:
policyurl = baseurl + "/admin/v1/Apps/" + targetappid + "?attributes=urn:ietf:params:scim:schemas:oracle:idcs:extension:webTierPolicy:App:webTierPolicyJson"

currentpolicy = requests.get(policyurl, headers = appsheaders, verify=False).json().get("urn:ietf:params:scim:schemas:oracle:idcs:extension:webTierPolicy:App", None)

if(currentpolicy):
    policyjson = currentpolicy["webTierPolicyJson"]
    print("Current Policy:")
    print(policyjson)

Specify the target Policy

In [ ]:
targetpolicy = None

def getpolicy(pol):
    global targetpolicy
    if(pol):
        targetpolicy = json.loads(pol);

interact(getpolicy, pol=widgets.Textarea(description='Target Policy:', layout=Layout(width='60%', height='300px')))

In [ ]:
validatorUrl = baseurl  + "/admin/v1/WebTierPolicyJsonValidator"

valdata = {
    "webTierPolicyJson": json.dumps(targetpolicy),
    "schemas": ["urn:ietf:params:scim:schemas:oracle:idcs:WebTierPolicyJsonValidator"]
}

response = requests.post(validatorUrl, data = json.dumps(valdata), headers = appsheaders, verify=False).json()

if response.get("status", None) != None:
    print("Policy Validation Failed. Rectify the Target Policy in previous step and try again")
    print(response.get("detail", None))
else:
    apppatchurl = baseurl + "/admin/v1/Apps/" + targetappid
    patchpayload = {
    "schemas": ["urn:ietf:params:scim:api:messages:2.0:PatchOp"],
     "Operations": [{
          "op"    : "replace",
          "path"  : "isWebTierPolicy",
          "value" :  True
     },{
          "op"    : "replace",
          "path"  : "urn:ietf:params:scim:schemas:oracle:idcs:extension:webTierPolicy:App:webTierPolicyJson",
          "value" : json.dumps(targetpolicy)
     }]
    }
    response = requests.patch(apppatchurl, data = json.dumps(patchpayload), headers = appsheaders, verify=False)
    print("Application policy setup done")
    


___

In [ ]:
policyurl = baseurl + "/admin/v1/Apps/" + targetappid + "?attributes=urn:ietf:params:scim:schemas:oracle:idcs:extension:webTierPolicy:App:webTierPolicyJson"

currentpolicy = requests.get(policyurl, headers = appsheaders, verify=False).json().get("urn:ietf:params:scim:schemas:oracle:idcs:extension:webTierPolicy:App", None)

if(currentpolicy):
    policyjson = currentpolicy["webTierPolicyJson"]
    print("Current Policy:")
    print(policyjson)